[View in Colaboratory](https://colab.research.google.com/github/assaflehr/Kaggle_Carvana_Segmentation/blob/master/notebooks/paraphraser_pytorch_quora.ipynb)

In [6]:
!pip install torch -U  # 0.4 at-least, on windows, read installation instrcution of pytorch.org
!pip install --quiet torchtext  # for simpler datasets
!pip install --quiet git+https://github.com/IBM/pytorch-seq2seq  #for seq2seq
!pip install --quiet dill  #req of seq2seq
!pip install --quiet tqdm  #req of seq2seq
!pip install --quiet revtok #for torchtext word tokenizer

#quora dataset
!pip install kaggle
import os
os.environ['KAGGLE_USERNAME'] = "myusername"
os.environ['KAGGLE_KEY'] = "mykey"
!kaggle competitions download -c quora-question-pairs
!unzip train.csv.zip







Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (0.4.1)
  Cloning https://github.com/IBM/pytorch-seq2seq to /tmp/pip-req-build-dyh9g0kx
  Running setup.py bdist_wheel for seq2seq ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-b3d51p8m/wheels/98/b5/06/771c406b3ecc8ed34f07da72d7baf65b87e561bd9f808e91bd
Successfully built seq2seq
    100% |████████████████████████████████| 61kB 1.2MB/s 
    100% |████████████████████████████████| 235kB 3.7MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/44/2c/df/22a6eeb780c36c28190faef6252b739fdc47145fd87a6642d4
  Running setup.py bdist_wheel for python-slugify ... - done
  Stored in directory: /root/.cache/pip/wheels/e3/65/da/2045deea3098ed7471eca0e2460cfbd3fdfe8c1d6fa6fcac92
Successfully built kaggle python-slugify
 88% 153M/173M [00:05<00:00, 25.6MB/s]
100% 173M/173M [00:05<00:00, 31.0MB/s]
100% 4.95M/4.95M [00:00<00:00, 24.3MB/s]

 79% 

In [7]:
!rm -r paraphraser_pytorch  #remove previous github copy if needed
!git clone https://github.com/assaflehr/paraphraser_pytorch.git
import sys
sys.path.append('paraphraser_pytorch/paraph')

%load_ext autoreload
%autoreload 2


Cloning into 'paraphraser_pytorch'...
remote: Counting objects: 96, done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 96 (delta 47), reused 76 (delta 27), pack-reused 0
Unpacking objects: 100% (96/96), done.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#option 1 : Run from command-line

In [9]:
! python paraphraser_pytorch/paraph/train.py --help

! python paraphraser_pytorch/paraph/train.py --dataset quora

usage: train.py [-h] [--dataset DATASET] [--epocs EPOCS]
                [--epoch_size EPOCH_SIZE] [--batch_size BATCH_SIZE]
                [--optimizer OPTIMIZER] [--lr LR] [--adv_disc_lr ADV_DISC_LR]
                [--beta1 BETA1] [--semantics_dim SEMANTICS_DIM]
                [--style_dim STYLE_DIM] [--sd_weight SD_WEIGHT]
                [--sem_sim_weight SEM_SIM_WEIGHT]
                [--max_sent_len MAX_SENT_LEN]

optional arguments:
  -h, --help            show this help message and exit
  --dataset DATASET     dataset to use. valid values are quora or bible
  --epocs EPOCS         number of epochs to train for
  --epoch_size EPOCH_SIZE
                        epoch size
  --batch_size BATCH_SIZE
                        batch size
  --optimizer OPTIMIZER
                        optimizer to train with. only Adam supported.
  --lr LR               learning rate. 0.001 is a food value
  --adv_disc_lr ADV_DISC_LR
                        learning rate
  --beta1 BETA1         mom

#Option 2 run from code directly

In [0]:
from train import train_main
from options import get_options

sys.argv= ["first_is_filename","--dataset","quora","--sem_sim_weight","0.01","--sd_weight","10","--epocs","100","--lr","0.001"] #change if needed
bucket_iter_train, bucket_iter_val, models= train_main()
#now you can query the iterators, look inside the model summary, etc 


running train with options: Namespace(adv_disc_lr=0.01, batch_size=32, beta1=0.5, dataset='quora', epoch_size=500, epocs=100, lr=0.001, max_sent_len=30, optimizer='adam', sd_weight=10.0, sem_sim_weight=0.01, semantics_dim=256, style_dim=100)
dups 149650, not_dups 413109
<class 'list'> <class 'float'> OneSample(sent_0=[' astrology ', ': ', ' i ', ' am ', ' a ', ' capricorn ', ' sun ', ' cap ', ' moon ', ' and ', ' cap ', ' rising ', '. ', '. ', '.', ' what ', ' does ', ' that ', ' say ', ' about ', ' me ', '? '], sent_1=[' i ', "'", ' m ', ' a ', ' triple ', ' capricorn ', ' (', ' sun ', ', ', ' moon ', ' and ', ' ascendant ', ' in ', ' capricorn ', ') ', ' what ', ' does ', ' this ', ' say ', ' about ', ' me ', '? '], sent_x=[' i ', ' want ', ' to ', ' start ', ' writing ', '. ', ' how ', ' do ', ' i ', ' begin ', '? '], is_x_0=1.0, sent_0_target=['<sos>', ' astrology ', ': ', ' i ', ' am ', ' a ', ' capricorn ', ' sun ', ' cap ', ' moon ', ' and ', ' cap ', ' rising ', '. ', '. ', '.'

09:07:45 INFO:Loading vectors from .vector_cache/wiki.simple.vec.pt


vocab TEXT: len 29464 common [('? ', 155779), ('<sos>', 149650), ('<eos>', 149650), (' the ', 64718), (' what ', 62187), (' is ', 48178), (' how ', 40218), (' i ', 33919), (' a ', 32373), (' to ', 32224)]
vocab TEXT_TARGET: 29464 [('? ', 155779), ('<sos>', 149650), ('<eos>', 149650), (' the ', 64718), (' what ', 62187), (' is ', 48178), (' how ', 40218), (' i ', 33919), (' a ', 32373), (' to ', 32224)]
vocab  <sos> 2 2
vocab  <eos> 3 3
vocab  out-of-vocab 3 0
training with lr 0.001


09:10:28 INFO:[00] train rec loss: 4.0046 | sim loss: 0.0064 | anti_disc_loss: 0.0002 || scene disc 0.6929 51.131% 
09:11:40 INFO:[00] eval rec loss: 3.5195 | sim loss: 0.0054 | anti_disc_loss: 0.0003 || scene disc 0.6928 52.663% 



              sent0:  what  is  the  best  way  to  teach  kids  alphabets ? <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
         sent0_targ: <sos><sos> what  is  the  best  way  to  teach  kids  alphabets ? <eos><eos><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
              sent1:  how  can  i  teach  my  kids  the  alphabet ? <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
              sentX:  where  can  i  find  a  video  of  steve  jobs  announcing  the  iphone ? <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
recon_sem0_sty0:[TF=0] <sos> what  is  the  best  way  to  learn  a  job ? <eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos>
recon_semX_sty0:[TF=0] <sos> what  is  the  best  way  to  learn  a  job ? <eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos>
recon_semX_sty1:[TF=0] <sos> how  can  i  get  a  

...


              sent0:  what  do  you  think  about  quora ? <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
         sent0_targ: <sos><sos> what  do  you  think  about  quora ? <eos><eos><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
              sent1:  what  do  you  think  about  quora  in  general ? <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
              sentX:  how  can  i  fix  my  laptop  that  won ' t  turn  on  anymore ? <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
recon_sem0_sty0:[TF=0] <sos> what  do  you  think  about  quora ? <eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos>
recon_semX_sty0:[TF=0] <so

10:12:17 INFO:[21] train rec loss: 1.7303 | sim loss: 0.0132 | anti_disc_loss: 0.1097 || scene disc 0.6593 63.888% 
10:14:54 INFO:[22] train rec loss: 1.6698 | sim loss: 0.0133 | anti_disc_loss: 0.1104 || scene disc 0.6596 63.731% 
10:17:32 INFO:[23] train rec loss: 1.6634 | sim loss: 0.0136 | anti_disc_loss: 0.1300 || scene disc 0.6646 61.906% 
10:20:11 INFO:[24] train rec loss: 1.6703 | sim loss: 0.0139 | anti_disc_loss: 0.1420 || scene disc 0.6658 61.612% 
10:22:49 INFO:[25] train rec loss: 1.6075 | sim loss: 0.0141 | anti_disc_loss: 0.1230 || scene disc 0.6596 63.644% 
10:25:27 INFO:[26] train rec loss: 1.6048 | sim loss: 0.0143 | anti_disc_loss: 0.1186 || scene disc 0.6624 62.712% 
10:28:06 INFO:[27] train rec loss: 1.5696 | sim loss: 0.0145 | anti_disc_loss: 0.1568 || scene disc 0.6652 61.825% 
10:30:45 INFO:[28] train rec loss: 1.5490 | sim loss: 0.0145 | anti_disc_loss: 0.1175 || scene disc 0.6571 63.925% 
10:33:26 INFO:[29] train rec loss: 1.5699 | sim loss: 0.0146 | anti_disc


              sent0:  should  one  clear  credit  card  debt  or  shiuld  give  down  payment  for  a  first  house ? <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
         sent0_targ: <sos><sos> should  one  clear  credit  card  debt  or  shiuld  give  down  payment  for  a  first  house ? <eos><eos><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
              sent1:  should  one  clear  credit  card  debt  or  should  give  down  payment  for  a  first  house ? <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
              sentX:  which  startups  are  hiring  in  mumbai ? <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
recon_sem0_sty0:[TF=0] <sos> should  i  buy  a  year - up  to  customer  service  or  will  i  purchase  for  a  business / 5 lakhs ? <eos><eos><eos><eos><eos><eos><eos><e

10:40:27 INFO:[31] train rec loss: 1.5524 | sim loss: 0.0151 | anti_disc_loss: 0.1738 || scene disc 0.6651 61.469% 
10:43:08 INFO:[32] train rec loss: 1.5209 | sim loss: 0.0150 | anti_disc_loss: 0.1328 || scene disc 0.6645 61.775% 
10:45:46 INFO:[33] train rec loss: 1.4930 | sim loss: 0.0152 | anti_disc_loss: 0.1768 || scene disc 0.6611 63.200% 
10:48:28 INFO:[34] train rec loss: 1.4893 | sim loss: 0.0156 | anti_disc_loss: 0.2453 || scene disc 0.6669 60.569% 
10:51:08 INFO:[35] train rec loss: 1.5092 | sim loss: 0.0154 | anti_disc_loss: 0.2491 || scene disc 0.6742 59.806% 
10:53:45 INFO:[36] train rec loss: 1.4694 | sim loss: 0.0158 | anti_disc_loss: 0.1944 || scene disc 0.6675 61.462% 
10:56:20 INFO:[37] train rec loss: 1.4492 | sim loss: 0.0152 | anti_disc_loss: 0.1818 || scene disc 0.6675 60.606% 
10:59:00 INFO:[38] train rec loss: 1.4580 | sim loss: 0.0154 | anti_disc_loss: 0.1556 || scene disc 0.6626 62.487% 
11:01:42 INFO:[39] train rec loss: 1.4314 | sim loss: 0.0157 | anti_disc


              sent0:  which  is  the  best  mouse  under  rs  1000 ? <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
         sent0_targ: <sos><sos> which  is  the  best  mouse  under  rs  1000 ? <eos><eos><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
              sent1:  which  is  the  best  mouse  i  can  get  for  under  rs . 1000 ? <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
              sentX:  what  is  the  process  of  admissions  for  indians  who  want  to  get  into  mit ? <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
recon_sem0_sty0:[TF=0] <sos> which  is  the  best  earphones  under  rs  1000  rs  500  and  rs  1000  notes ? <eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos>